In [1]:
# Import all the libraries required

import soccerdata as sd
import pandas as pd
from tqdm import tqdm
import ScraperFC as sfc
import traceback

import warnings

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
import pyspark.pandas as ps


[01/16/24 23:01:41] INFO     No custom team name replacements found. You can configure these in       ]8;id=429682;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=461414;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_config.py#84\84]8;;\
                             C:\Users\ksbha\soccerdata\config\teamname_replacements.json.                          

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=826753;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=79806;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_config.py#162\162]8;;\
                             C:\Users\ksbha\soccerdata\config\league_dict.json.                                    

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
## Create a connection to Snowflake
from sqlalchemy import create_engine
import snowflake.connector

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (14.0.2), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(


[01/16/24 23:01:42] WARNING  Failed to import ArrowResult. No Apache Arrow result set format can be    ]8;id=36912;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=97174;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#94\94]8;;\
                             used. ImportError: DLL load failed while importing arrow_iterator: The                
                             specified procedure could not be found.                                               

In [3]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

In [4]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Gegenstats") \
    .getOrCreate()

In [5]:
## Set options to view all columns
pd.set_option('display.max_columns', None)

In [6]:
# spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [7]:
scraper = sfc.Understat()
def scrape_league_data(years):
    output_df = pd.DataFrame(columns=['№', 'Team', 'M', 'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'NPxG', 'xGA',
       'NPxGA', 'NPxGD', 'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS', 'season','league'])
    for league_year in league_years:
        try:
            ooper = scraper.scrape_league_table(year=league_year[1], league=league_year[0])
            season_val = league_year[1]
            ooper['season'] = int(str(int(str(season_val)[2:])-1)+str(season_val)[2:])
            if league_year[0] == 'EPL':
                ooper['league'] = 'ENG-Premier League'
            output_df = pd.concat([output_df, ooper])
            
        except Exception as e:
            print(f"An error occurred for the year {league_years}: {e}")
            traceback.print_exc()
    scraper.close()
    return output_df

league_years = [('EPL', 2023)]  # Replace with your list of years
out = scrape_league_data(league_years)

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\ScraperFC\Understat.py:361: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  columns = pd.read_html(table)[0].columns
d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\ScraperFC\Understat.py:366: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table)[0]
C:\Users\ksbha\AppData\Local\Temp\ipykernel_3300\1976754889.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output_df = pd.concat([output_df, ooper])


In [8]:
understat_table = out[['league','Team','season', 'W', 'D', 'L','G','GA', 'xG', 'xGA','NPxG','NPxGA','PPDA',
                       'OPPDA','PTS','xPTS']]

In [9]:
understat_table.loc[understat_table['Team'] == 'Manchester United', 'Team'] = 'Manchester Utd'
understat_table.loc[understat_table['Team'] == 'Newcastle United', 'Team'] = 'Newcastle Utd'
understat_table.loc[understat_table['Team'] == 'Tottenham Hotspur', 'Team'] = 'Tottenham'
understat_table.loc[understat_table['Team'] == 'Wolverhampton Wanderers', 'Team'] = 'Wolves'
understat_table.loc[understat_table['Team'] == 'Nottingham Forest', 'Team'] = "Nott'ham Forest"
understat_table.loc[understat_table['Team'] == 'Leicester', 'Team'] = 'Leicester City'
understat_table.loc[understat_table['Team'] == 'Leeds', 'Team'] = 'Leeds United'
understat_table.loc[understat_table['Team'] == 'Sheffield United', 'Team'] = "Sheffield Utd"
understat_table.loc[understat_table['Team'] == 'Luton Town', 'Team'] = 'Luton'
understat_table.loc[understat_table['Team'] == 'Norwich', 'Team'] = 'Norwich City'

In [10]:
understat_table.sort_values(by=['league','season','Team'], ascending=True, inplace=True)
understat_table.reset_index(drop=True, inplace=True)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_3300\614021855.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  understat_table.sort_values(by=['league','season','Team'], ascending=True, inplace=True)


In [11]:
understat_table.rename(columns={'Team':'TEAM_NAME', 'W':'TEAM_WINS', 'D':'TEAM_DRAWS', 'league':'COMPETITION',
                                'L': 'TEAM_LOSSES', 'xPTS':'TEAM_XPTS', 'PTS':'TEAM_PTS', 'season':'SEASON',
                                'NPxGA':'NPXG_AGAINST', 'xGA':'XG_AGAINST'}, inplace=True)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_3300\2868126508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  understat_table.rename(columns={'Team':'TEAM_NAME', 'W':'TEAM_WINS', 'D':'TEAM_DRAWS', 'league':'COMPETITION',


In [12]:
understat_table.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [99]:
fbref = sd.FBref(leagues="ENG-Premier League", seasons=[2223])
fbref_team_standard = fbref.read_team_season_stats(stat_type='standard')
fbref_team_standard_oppo = fbref.read_team_season_stats(stat_type='standard', opponent_stats=True)
fbref_team_shooting = fbref.read_team_season_stats(stat_type='shooting')
fbref_team_passing = fbref.read_team_season_stats(stat_type='passing')
fbref_team_defense = fbref.read_team_season_stats(stat_type='defense')
fbref_team_possession = fbref.read_team_season_stats(stat_type='possession')
fbref_team_misc = fbref.read_team_season_stats(stat_type='misc')
fbref_schedule = fbref.read_schedule()
# fbref_lineups = fbref.read_lineup()

[12/21/23 13:49:05] INFO     Saving cached data to C:\Users\ksbha\soccerdata\data\FBref               ]8;id=324650;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=159865;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py#91\91]8;;\

In [175]:
fbref_team_standard = fbref_team_standard.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_standard_oppo = fbref_team_standard_oppo.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_shooting = fbref_team_shooting.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_passing = fbref_team_passing.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_defense = fbref_team_defense.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_possession = fbref_team_possession.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_misc = fbref_team_misc.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_schedule = fbref_schedule.rename_axis(index={'season': 'SEASON', 'league': 'COMPETITION'})
# fbref_lineups = fbref_lineups.rename_axis(index={'season': 'SEASON', 'league': 'COMPETITION'})

In [177]:
def make_season_integer(df):
    indexes = df.index.names
    df = df.reset_index()
    df['SEASON'] = df['SEASON'].astype(int)
    df = df.set_index(indexes)
    return df

fbref_team_standard = make_season_integer(fbref_team_standard)
fbref_team_standard_oppo = make_season_integer(fbref_team_standard_oppo)
fbref_team_shooting = make_season_integer(fbref_team_shooting)
fbref_team_passing = make_season_integer(fbref_team_passing)
fbref_team_defense = make_season_integer(fbref_team_defense)
fbref_team_possession = make_season_integer(fbref_team_possession)
fbref_team_misc = make_season_integer(fbref_team_misc)
fbref_schedule = make_season_integer(fbref_schedule)
# fbref_lineups = make_season_integer(fbref_lineups)

In [16]:
team_matches_played = fbref_team_standard['Playing Time'].MP
team_goals_scored = understat_table.G
team_npxG_90 = fbref_team_standard['Per 90 Minutes'].npxG
team_goals_conceded = understat_table.GA
team_xG_against_90 = fbref_team_standard_oppo['Per 90 Minutes'].xG
team_shots = fbref_team_shooting['Standard'].Sh
team_shotsOT = fbref_team_shooting['Standard'].SoT
team_pass_completed = fbref_team_passing["Total"].Cmp
team_pass_attempted = fbref_team_passing["Total"].Att
team_tackles = fbref_team_defense.Tackles.Tkl
team_tackles_won = fbref_team_defense.Tackles.TklW
team_takeons_attempted = fbref_team_possession['Take-Ons'].Att
team_takeons_completed = fbref_team_possession['Take-Ons'].Succ
team_crossesintoPA = fbref_team_passing['CrsPA']
team_fouls_against = fbref_team_misc['Performance']['Fld']
team_fouls_made = fbref_team_misc['Performance']['Fls']
team_interceptions = fbref_team_defense['Int']
team_blocks_shots = fbref_team_defense['Blocks'].Sh
team_blocks_pass = fbref_team_defense['Blocks'].Pass
team_clearances = fbref_team_defense.Clr
team_aerials_won = fbref_team_misc['Aerial Duels'].Won
team_aerials_lost = fbref_team_misc['Aerial Duels'].Lost

In [17]:
team_names = fbref_team_standard.reset_index()[['TEAM_NAME','url']]
team_names['TEAM_FBREF_ID'] = team_names['url'].apply(lambda x: x.split('/')[3])      ## Extracting team_id from URL
team_names.drop(columns=['url'], inplace=True)
team_names.columns = team_names.columns.droplevel(1)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_3300\2026419047.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  team_names.drop(columns=['url'], inplace=True)


In [18]:
team_names['TEAM_FBREF_ID'] = team_names['TEAM_FBREF_ID'].astype('string')

In [19]:
SNOWFLAKE_USER = 'kbharaj3'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'qx25653.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'FOOTY_STORE'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'TABLES'

In [20]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
    )

[12/21/23 13:29:48] INFO     Snowflake Connector for Python Version: 3.3.1, Python Version:       ]8;id=896165;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py\connection.py]8;;\:]8;id=424615;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py#334\334]8;;\
                             3.11.5, Platform: Windows-10-10.0.22000-SP0                                           

                    INFO     This connection is in OCSP Fail Open Mode. TLS Certificates would   ]8;id=868927;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py\connection.py]8;;\:]8;id=40382;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py#1103\1103]8;;\
                             be checked for validity and revocation status. Any other                              
                             Certificate Revocation related exceptions or OCSP Responder                           
                             failures would be disregarded in favor of connectivity.                               

                    INFO     Setting use_openssl_only mode to False                              ]8;id=892556;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py\connection.py]8;;\:]8;id=175573;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py#1121\1121]8;;\

In [21]:
cur = conn.cursor()
cur.execute(f"USE WAREHOUSE {SNOWFLAKE_WAREHOUSE}")

[12/21/23 13:29:49] INFO     query: [USE WAREHOUSE FOOTY_STORE]                                       ]8;id=40890;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=13200;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=239579;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=116397;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=873227;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=298846;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [22]:
create_schema_sql = f"CREATE SCHEMA IF NOT EXISTS {SNOWFLAKE_SCHEMA}"
cur.execute(create_schema_sql)

[12/21/23 13:29:51] INFO     query: [CREATE SCHEMA IF NOT EXISTS TABLES]                              ]8;id=740362;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=84348;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=979722;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=349187;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=815464;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=359929;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [23]:
print("The Snowflake warehouse being used is:", SNOWFLAKE_WAREHOUSE)

The Snowflake warehouse being used is: FOOTY_STORE


In [24]:
create_table_query = """
CREATE TABLE IF NOT EXISTS TEAMS (
	TEAM_NAME VARCHAR(255),
	TEAM_FBREF_ID VARCHAR(255) PRIMARY KEY,
    TEAM_LOGO_URL VARCHAR(255)
);
"""

In [25]:
cur.execute(create_table_query)

[12/21/23 13:30:06] INFO     query: [CREATE TABLE IF NOT EXISTS TEAMS ( TEAM_NAME VARCHAR(255),       ]8;id=582316;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=226437;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             TEAM_FBREF_ID VARCHAR...]                                                             

                    INFO     query execution done                                                     ]8;id=213779;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=183941;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=363450;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=840646;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [26]:
def insert_to_snowflake_DB(primary_keys, table_name, dataframe, replace=False):
    if replace:
        cur.execute(f"TRUNCATE TABLE {table_name}")
        success, nchunks, nrows, _ = write_pandas(conn, dataframe, table_name)
    else:
        cur.execute(f"SELECT {', '.join(primary_keys)} FROM {table_name}")
        existing_ids = cur.fetchall()
        existing_ids = [id for id in existing_ids]
        dataframe.set_index(primary_keys, inplace=True)
        new_rows = dataframe[~dataframe.index.isin(existing_ids)]
        new_rows.reset_index(inplace=True)
        dataframe.reset_index(inplace=True)
        if not new_rows.empty:
            success, nchunks, nrows, _ = write_pandas(conn, new_rows, table_name)
            print(f"Inserted {nrows} new rows")
        else:
            print("No new rows to insert")

In [27]:
team_logos = \
[["18bb7c10", "https://i.imgur.com/SURo5sj.png"], ["8602292d", "https://i.imgur.com/M4mwH1X.png"],
 ["4ba7cbea", "https://i.imgur.com/0F6UIO4.png"], ["cd051869", "https://i.imgur.com/MOhGOQ4.png"],
 ["d07537b9", "https://i.imgur.com/EMCwD3X.png"], ["cff3d9bb", "https://i.imgur.com/6F7h3UR.png"],
 ["47c64c55", "https://i.imgur.com/LPhzr0K.png"], ["d3fd31cc", "https://i.imgur.com/Dgei0uj.png"],
 ["fd962109", "https://i.imgur.com/2zggSrF.png"], ["5bfb9659", "https://i.imgur.com/2tipH85.png"],
 ["a2d435b3", "https://i.imgur.com/ldOUO84.png"], ["822bd0ba", "https://i.imgur.com/fs7VK9G.png"],
 ["b8fd03ef", "https://i.imgur.com/FF8z7uZ.png"], ["19538871", "https://i.imgur.com/z47emBQ.png"],
 ["b2b47a98", "https://i.imgur.com/3kOLokZ.png"], ["e4a775cb", "https://i.imgur.com/z4Tl9Zu.png"],
 ["33c895d4", "https://i.imgur.com/35GrgJC.png"], ["361ca564", "https://i.imgur.com/fJ7rUWW.png"],
 ["7c21e445", "https://i.imgur.com/9BQHJRl.png"], ["8cec06e1", "https://i.imgur.com/6azSF88.png"]
 ]

In [28]:
df_logos = pd.DataFrame(team_logos, columns=['TEAM_FBREF_ID', 'TEAM_LOGO'])

In [29]:
team_names = team_names.merge(df_logos, on='TEAM_FBREF_ID', how='left')

In [30]:
team_names.rename(columns={'TEAM_LOGO':'TEAM_LOGO_URL'}, inplace=True)

In [31]:
insert_to_snowflake_DB(['TEAM_FBREF_ID'], 'TEAMS', team_names, replace=True)

[12/21/23 13:30:37] INFO     query: [TRUNCATE TABLE TEAMS]                                            ]8;id=854565;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=813028;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=644731;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=326127;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=201089;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=43220;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=589981;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=943573;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=112185;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=211427;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=572709;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=971327;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=601076;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=656552;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

[12/21/23 13:30:38] INFO     query: [COPY INTO "TEAMS" /*                                             ]8;id=473013;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=549027;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ (...]                       

[12/21/23 13:30:39] INFO     query execution done                                                     ]8;id=448851;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=442328;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=392279;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=109759;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [32]:
team_names.drop(columns=['TEAM_LOGO_URL'], inplace=True)

In [34]:
create_standard_team_stats_query = """
CREATE TABLE IF NOT EXISTS TEAM_STANDARD_STATS (
    TEAM_FBREF_ID VARCHAR(255),
    SEASON INT,
    COMPETITION VARCHAR(255),
    MATCHES_PLAYED INT,
    TEAM_WINS INT,
    TEAM_DRAWS INT,
    TEAM_LOSSES INT,
    TEAM_PTS INT,
    TEAM_XPTS DOUBLE,
    PRIMARY KEY (TEAM_FBREF_ID, COMPETITION, SEASON),
    FOREIGN KEY (TEAM_FBREF_ID) REFERENCES GEGENSTATS.TABLES.TEAMS(TEAM_FBREF_ID)
);
"""

In [35]:
cur.execute(create_standard_team_stats_query)

[12/21/23 13:31:18] INFO     query: [CREATE TABLE IF NOT EXISTS TEAM_STANDARD_STATS ( TEAM_FBREF_ID   ]8;id=554412;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=164137;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             VARCHAR(255), SEA...]                                                                 

                    INFO     query execution done                                                     ]8;id=872119;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=289550;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=996888;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=507515;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [36]:
team_standard_stats = understat_table.copy()

In [37]:
team_standard_stats['MATCHES_PLAYED'] = team_matches_played

In [38]:
team_standard_stats.reset_index(inplace=True)

In [39]:
## Merge team_matches_played with team_names to get TEAM_ID
team_standard_stats = team_standard_stats.merge(team_names, on='TEAM_NAME', how='left')

In [40]:
team_standard_stats = team_standard_stats[['TEAM_FBREF_ID','SEASON','COMPETITION','MATCHES_PLAYED','TEAM_WINS', 
                                           'TEAM_DRAWS', 'TEAM_LOSSES', 'TEAM_PTS', 'TEAM_XPTS']]

In [41]:
insert_to_snowflake_DB(['TEAM_FBREF_ID', 'COMPETITION','SEASON'], 'TEAM_STANDARD_STATS', team_standard_stats, replace=True)

[12/21/23 13:31:49] INFO     query: [TRUNCATE TABLE TEAM_STANDARD_STATS]                              ]8;id=808509;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=960217;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=123479;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=275304;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=529386;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=466442;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=355634;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=962132;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=352023;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=383031;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=928818;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=935210;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

[12/21/23 13:31:50] INFO     query execution done                                                     ]8;id=148764;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=957986;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [COPY INTO "TEAM_STANDARD_STATS" /*                               ]8;id=429665;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=896352;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write...]                                     

[12/21/23 13:31:51] INFO     query execution done                                                     ]8;id=338008;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=330623;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=241700;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=196821;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [42]:
create_team_attacking_stats_query = """
CREATE TABLE IF NOT EXISTS TEAM_ATTACKING_STATS (
    TEAM_FBREF_ID VARCHAR(255),
    SEASON INT,
    COMPETITION VARCHAR(255),
    GOALS_SCORED INT,
    NPXG DOUBLE,
    XG DOUBLE,
    SHOTS INT,
    SHOTS_ON_TARGET INT,
    PASS_COMPLETED INT,
    PASS_ATTEMPTED INT,
    TAKEONS_ATTEMPTED INT,
    TAKEONS_COMPLETED INT,
    CROSSES_INTO_PA INT,
    FOULS_AGAINST INT,
    PRIMARY KEY (TEAM_FBREF_ID, COMPETITION, SEASON),
    FOREIGN KEY (TEAM_FBREF_ID) REFERENCES GEGENSTATS.TABLES.TEAMS(TEAM_FBREF_ID)
);
"""

In [43]:
cur.execute(create_team_attacking_stats_query)

[12/21/23 13:32:29] INFO     query: [CREATE TABLE IF NOT EXISTS TEAM_ATTACKING_STATS ( TEAM_FBREF_ID  ]8;id=580731;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=226406;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             VARCHAR(255), SE...]                                                                  

                    INFO     query execution done                                                     ]8;id=949980;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=690092;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=908829;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=163003;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [44]:
team_attacking_stats = understat_table.copy()

In [45]:
team_attacking_stats = team_attacking_stats.reset_index().merge(team_names, on='TEAM_NAME', how='left')

In [46]:
team_attacking_stats = team_attacking_stats[['TEAM_NAME','TEAM_FBREF_ID','COMPETITION','SEASON','NPxG','xG']]

In [47]:
team_attacking_stats.head()

,TEAM_NAME,TEAM_FBREF_ID,COMPETITION,SEASON,NPxG,xG
0,Arsenal,18bb7c10,ENG-Premier League,2223,73.47,76.51
1,Aston Villa,8602292d,ENG-Premier League,2223,46.95,50.0
2,Bournemouth,4ba7cbea,ENG-Premier League,2223,40.08,40.08
3,Brentford,cd051869,ENG-Premier League,2223,53.14,59.23
4,Brighton,d07537b9,ENG-Premier League,2223,73.16,77.72


In [48]:
team_attacking_stats.rename(columns={'NPxG': 'NPXG', 'xG':'XG'}, inplace=True)

In [49]:
team_attacking_stats.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [50]:
team_attacking_stats['GOALS_SCORED'] = team_goals_scored
team_attacking_stats['SHOTS'] = team_shots
team_attacking_stats['SHOTS_ON_TARGET'] = team_shotsOT
team_attacking_stats['PASS_COMPLETED'] = team_pass_completed
team_attacking_stats['PASS_ATTEMPTED'] = team_pass_attempted
team_attacking_stats['TAKEONS_ATTEMPTED'] = team_takeons_attempted
team_attacking_stats['TAKEONS_COMPLETED'] = team_takeons_completed
team_attacking_stats['CROSSES_INTO_PA'] = team_crossesintoPA
team_attacking_stats['FOULS_AGAINST'] = team_fouls_against

In [51]:
team_attacking_stats.reset_index(inplace=True)
team_attacking_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [52]:
insert_to_snowflake_DB(['TEAM_FBREF_ID', 'COMPETITION','SEASON'], 'TEAM_ATTACKING_STATS', team_attacking_stats,replace=True)

[12/21/23 13:33:05] INFO     query: [TRUNCATE TABLE TEAM_ATTACKING_STATS]                             ]8;id=947319;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=650058;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=915234;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=472319;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=165104;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=379220;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=730248;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=486829;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=481271;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=170659;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=263619;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=637030;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

[12/21/23 13:33:06] INFO     query execution done                                                     ]8;id=278479;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=249435;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [COPY INTO "TEAM_ATTACKING_STATS" /*                              ]8;id=356077;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=693176;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.writ...]                                      

[12/21/23 13:33:07] INFO     query execution done                                                     ]8;id=469548;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=677618;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=29716;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=260550;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [7]:
create_team_defending_stats_query = """
CREATE TABLE IF NOT EXISTS TEAM_DEFENDING_STATS (
    TEAM_FBREF_ID VARCHAR(255),
    SEASON INT,
    COMPETITION VARCHAR(255),
    GOALS_CONCEDED INT,
    XG_AGAINST DOUBLE,
    NPXG_AGAINST DOUBLE,
    TACKLES INT,
    TACKLES_WON INT,
    FOULS_MADE INT,
    INTERCEPTIONS INT,
    BLOCKED_SHOTS INT,
    BLOCKED_PASSES INT,
    CLEARANCES INT,
    PPDA DOUBLE,
    OPP_PPDA DOUBLE,               -- New field for Opponent's PPDA
    FINAL_3RD_PASSES_AGAINST INT,  -- New field for Final Third Passes Against
    POSS_WON INT,                  -- New field for Possession Won
    CLEAN_SHEETS INT,              -- New field for Clean Sheets
    PRIMARY KEY (TEAM_FBREF_ID, COMPETITION, SEASON),
    FOREIGN KEY (TEAM_FBREF_ID) REFERENCES GEGENSTATS.TABLES.TEAMS(TEAM_FBREF_ID)
);
"""

In [8]:
cur.execute(create_team_defending_stats_query)

NameError: name 'cur' is not defined

In [80]:
team_defending_stats = understat_table.copy()

In [81]:
team_defending_stats = team_defending_stats.reset_index().merge(team_names, on='TEAM_NAME', how='left')

In [83]:
team_defending_stats = team_defending_stats[['TEAM_NAME','TEAM_FBREF_ID','COMPETITION','SEASON']]

In [84]:
team_defending_stats.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [85]:
team_defending_stats['GOALS_CONCEDED'] = team_goals_conceded
team_defending_stats['XG_AGAINST'] = understat_table['XG_AGAINST']
team_defending_stats['NPXG_AGAINST'] = understat_table['NPXG_AGAINST']
team_defending_stats['TACKLES'] = team_tackles
team_defending_stats['TACKLES_WON'] = team_tackles_won
team_defending_stats['FOULS_MADE'] = team_fouls_made
team_defending_stats['INTERCEPTIONS'] = team_interceptions
team_defending_stats['BLOCKED_SHOTS'] = team_blocks_shots
team_defending_stats['BLOCKED_PASSES'] = team_blocks_pass
team_defending_stats['CLEARANCES'] = team_clearances
team_defending_stats['PPDA'] = understat_table['PPDA']

In [86]:
team_defending_stats.reset_index(inplace=True)
team_defending_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [87]:
insert_to_snowflake_DB(['TEAM_FBREF_ID','COMPETITION','SEASON'], 'TEAM_DEFENDING_STATS', team_defending_stats,
                       replace=True)

[12/21/23 13:40:46] INFO     query: [TRUNCATE TABLE TEAM_DEFENDING_STATS]                             ]8;id=570610;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=52322;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

[12/21/23 13:40:47] INFO     query execution done                                                     ]8;id=136600;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=107605;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=8296;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=156362;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=3780;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=376342;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=936899;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=528674;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=924637;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=28463;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=807394;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=467476;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [COPY INTO "TEAM_DEFENDING_STATS" /*                              ]8;id=271492;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=884116;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.writ...]                                      

[12/21/23 13:40:50] INFO     query execution done                                                     ]8;id=729980;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=435759;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=796272;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=626876;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [206]:
create_team_misc_stats_query = """
CREATE TABLE IF NOT EXISTS TEAM_MISC_STATS (
    TEAM_FBREF_ID VARCHAR(255),
    SEASON INT,
    COMPETITION VARCHAR(255),
    AERIALS_WON INT,
    AERIALS_LOST INT,
    PRIMARY KEY (TEAM_FBREF_ID, COMPETITION, SEASON),
    FOREIGN KEY (TEAM_FBREF_ID) REFERENCES GEGENSTATS.TABLES.TEAMS(TEAM_FBREF_ID)
);
"""

In [207]:
cur.execute(create_team_misc_stats_query)

[12/21/23 17:34:20] INFO     query: [CREATE TABLE IF NOT EXISTS TEAM_MISC_STATS ( TEAM_FBREF_ID       ]8;id=980244;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=634507;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             VARCHAR(255), SEASON ...]                                                             

                    INFO     query execution done                                                     ]8;id=586014;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=186906;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=918885;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=930372;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [208]:
team_misc_stats = understat_table.copy()

In [209]:
team_misc_stats = team_misc_stats.reset_index().merge(team_names, on='TEAM_NAME', how='left')

In [211]:
team_misc_stats = team_misc_stats[['TEAM_NAME','TEAM_FBREF_ID','COMPETITION','SEASON']]

In [212]:
team_misc_stats.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [213]:
team_misc_stats['AERIALS_WON'] = team_aerials_won
team_misc_stats['AERIALS_LOST'] = team_aerials_lost

In [214]:
team_misc_stats.reset_index(inplace=True)
team_misc_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [215]:
insert_to_snowflake_DB(['TEAM_FBREF_ID', 'COMPETITION', 'SEASON'], 'TEAM_MISC_STATS', team_misc_stats, replace=True)

[12/21/23 17:34:28] INFO     query: [TRUNCATE TABLE TEAM_MISC_STATS]                                  ]8;id=587298;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=115386;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=639443;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=924678;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=203209;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=733752;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=205588;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=475959;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

[12/21/23 17:34:29] INFO     query execution done                                                     ]8;id=232649;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=570586;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=590233;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=756710;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=781804;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=77399;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [COPY INTO "TEAM_MISC_STATS" /*                                   ]8;id=799571;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=243406;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pan...]                                 

[12/21/23 17:34:30] INFO     query execution done                                                     ]8;id=80769;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=555242;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=449351;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=131625;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

# Now, inclusion of Match, Season and Competition data

In [114]:
create_competition_query = """
CREATE TABLE IF NOT EXISTS COMPETITIONS (
    COMPETITION VARCHAR(255),
    COMPETITION_NAME VARCHAR(255),
    COMPETITION_ACRONYM VARCHAR(255),
    SEASON INT,
    PRIMARY KEY (COMPETITION, SEASON)
);
"""

In [115]:
cur.execute(create_competition_query)

[12/21/23 13:55:26] INFO     query: [CREATE TABLE IF NOT EXISTS COMPETITIONS ( COMPETITION            ]8;id=949364;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=42512;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             VARCHAR(255), COMPETITION_...]                                                        

[12/21/23 13:55:27] INFO     query execution done                                                     ]8;id=522356;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=424761;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=983667;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=192691;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [116]:
df_competitions = fbref_schedule.reset_index()[['COMPETITION', 'SEASON']].drop_duplicates().copy()

In [117]:
df_competitions['COMPETITION_NAME'] = ''
df_competitions['COMPETITION_ACRONYM'] = ''

In [119]:
df_competitions.loc[df_competitions['COMPETITION'] == 'ENG-Premier League', 'COMPETITION_NAME'] = 'English Premier League'
df_competitions.loc[df_competitions['COMPETITION'] == 'ENG-Premier League', 'COMPETITION_ACRONYM'] = 'EPL'

In [122]:
insert_to_snowflake_DB(['COMPETITION','SEASON'], 'COMPETITIONS', df_competitions, replace=True)

[12/21/23 13:56:55] INFO     query: [TRUNCATE TABLE COMPETITIONS]                                     ]8;id=647267;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=888488;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=189827;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=671092;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=618402;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=425520;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

C:\Users\ksbha\AppData\Local\Temp\ipykernel_3300\2303712821.py:4: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, _ = write_pandas(conn, dataframe, table_name)


                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=655025;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=748301;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

...

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=166136;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=123548;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

[12/21/23 13:56:56] INFO     query execution done                                                     ]8;id=860222;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=583939;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [COPY INTO "COMPETITIONS" /*                                      ]8;id=404491;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=290314;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas...]                              

[12/21/23 13:56:57] INFO     query execution done                                                     ]8;id=377990;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=349459;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=149476;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=472685;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [123]:
create_season_query = """
CREATE TABLE IF NOT EXISTS SEASONS (
	SEASON INT PRIMARY KEY,
    SEASON_NAME VARCHAR(255),
    START_YEAR INT,
    END_YEAR INT
);
"""

In [124]:
cur.execute(create_season_query)

[12/21/23 13:57:31] INFO     query: [CREATE TABLE IF NOT EXISTS SEASONS ( SEASON INT PRIMARY KEY,     ]8;id=250514;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=62320;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             SEASON_NAME VARCHAR...]                                                               

                    INFO     query execution done                                                     ]8;id=648682;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=290848;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=131065;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=823162;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [134]:
df_seasons = df_competitions[['SEASON']].copy()

In [135]:
df_seasons['SEASON_NAME'] = ''
df_seasons['START_YEAR'] = 0
df_seasons['END_YEAR'] = 0

In [137]:
df_seasons.loc[df_seasons['SEASON'] == 2223, 'SEASON_NAME'] = '2022-2023'
df_seasons.loc[df_seasons['SEASON'] == 2223, 'START_YEAR'] = 2022
df_seasons.loc[df_seasons['SEASON'] == 2223, 'END_YEAR'] = 2023

In [139]:
insert_to_snowflake_DB(['SEASON'], 'SEASONS', df_seasons, replace=True)

[12/21/23 14:03:01] INFO     query: [TRUNCATE TABLE SEASONS]                                          ]8;id=920210;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=958185;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=150889;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=816013;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=202433;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=933508;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

C:\Users\ksbha\AppData\Local\Temp\ipykernel_3300\2303712821.py:4: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, _ = write_pandas(conn, dataframe, table_name)


                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=33095;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=880756;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

...

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=685002;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=169402;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=794768;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=703955;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

[12/21/23 14:03:02] INFO     query: [COPY INTO "SEASONS" /*                                           ]8;id=520487;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=400162;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */...]                         

[12/21/23 14:03:03] INFO     query execution done                                                     ]8;id=141857;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=342262;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=43540;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=858385;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [201]:
create_stadium_query = """
CREATE TABLE IF NOT EXISTS STADIUMS (
	STADIUM VARCHAR(255),
    TEAM_FBREF_ID VARCHAR(255),
    SEASON INT,
    PRIMARY KEY (TEAM_FBREF_ID, STADIUM, SEASON)
);
"""

In [202]:
cur.execute(create_stadium_query)

[12/21/23 17:32:42] INFO     query: [CREATE TABLE IF NOT EXISTS STADIUMS ( STADIUM VARCHAR(255),      ]8;id=231939;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=808933;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             TEAM_FBREF_ID VARCHA...]                                                              

                    INFO     query execution done                                                     ]8;id=127130;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=225416;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=432970;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=361583;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [203]:
df_stadiums = fbref_schedule.reset_index()[['home_team', 'SEASON',
            'venue']].drop_duplicates().rename(columns={'home_team': 'TEAM_NAME', 'venue': 'STADIUM'})

In [204]:
df_stadiums = df_stadiums.merge(team_names, on='TEAM_NAME')[['STADIUM','SEASON','TEAM_FBREF_ID']]

In [205]:
insert_to_snowflake_DB(['TEAM_FBREF_ID','STADIUM', 'SEASON'], 'STADIUMS', df_stadiums, replace=True)

[12/21/23 17:33:06] INFO     query: [TRUNCATE TABLE STADIUMS]                                         ]8;id=437989;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=320760;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=975821;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=267721;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=252517;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=746538;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=642122;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=820469;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=619342;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=887091;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=750989;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=138469;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=747599;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=351039;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

[12/21/23 17:33:07] INFO     query: [COPY INTO "STADIUMS" /*                                          ]8;id=422545;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=684440;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() *...]                          

[12/21/23 17:33:08] INFO     query execution done                                                     ]8;id=425271;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=530241;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=57890;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=544334;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [171]:
create_match_query = """
CREATE TABLE IF NOT EXISTS MATCHES (
    MATCH_ID VARCHAR(255) PRIMARY KEY,
    DATE_TIME VARCHAR(255), 
    HOME_TEAM_ID VARCHAR(255),
    AWAY_TEAM_ID VARCHAR(255),
    COMPETITION VARCHAR(255),
    SEASON INT,
    STADIUM VARCHAR(255),
    GAMEWEEK INT,
    DAY VARCHAR(255),
    HOME_TEAM_SCORE INT,
    AWAY_TEAM_SCORE INT,
    HOME_TEAM_XG DOUBLE,
    AWAY_TEAM_XG DOUBLE,
    ATTENDANCE INT,
    REFEREE VARCHAR(255),
    FOREIGN KEY (HOME_TEAM_ID) REFERENCES GEGENSTATS.TABLES.TEAMS(TEAM_FBREF_ID),
    FOREIGN KEY (AWAY_TEAM_ID) REFERENCES GEGENSTATS.TABLES.TEAMS(TEAM_FBREF_ID),
    FOREIGN KEY (COMPETITION, SEASON) REFERENCES GEGENSTATS.TABLES.COMPETITIONS(COMPETITION, SEASON),
    FOREIGN KEY (SEASON) REFERENCES GEGENSTATS.TABLES.SEASONS(SEASON),
    FOREIGN KEY (HOME_TEAM_ID, STADIUM) REFERENCES GEGENSTATS.TABLES.STADIUMS(TEAM_FBREF_ID, STADIUM)
);
"""

In [172]:
cur.execute(create_match_query)

[12/21/23 14:21:01] INFO     query: [CREATE TABLE IF NOT EXISTS MATCHES ( MATCH_ID VARCHAR(255)       ]8;id=268874;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=270482;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             PRIMARY KEY, DATE_TIM...]                                                             

                    INFO     query execution done                                                     ]8;id=438388;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=280612;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=841568;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=230134;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [183]:
df_matches = fbref_schedule.reset_index()[['game_id', 'date', 'time', 'home_team', 'away_team', 'COMPETITION', 'SEASON', 'venue',
                                           'week', 'day', 'score', 'home_xg', 'away_xg', 'attendance', 'referee']]

In [184]:
df_matches = df_matches.merge(team_names.rename(columns={'TEAM_FBREF_ID': 'home_team_id', 
                                                         'TEAM_NAME': 'home_team'}), on='home_team', how='left')

In [185]:
df_matches = df_matches.merge(team_names.rename(columns={'TEAM_FBREF_ID': 'away_team_id', 
                                                         'TEAM_NAME': 'away_team'}), on='away_team', how='left')

In [186]:
# Combine df_matches date and time to a dt.datatime configuration
df_matches['date'] = df_matches['date'].astype(str)
df_matches['time'] = df_matches['time'].astype(str)
df_matches['date_time'] = df_matches['date'] + ' ' + df_matches['time']
df_matches['date_time'] = pd.to_datetime(df_matches['date_time'])

In [187]:
# Split score in df_matches by "-" to home_team_score and away_team_score 
df_matches[['home_team_score', 'away_team_score']] = df_matches['score'].str.split('–', expand=True)

In [192]:
df_matches.head()

,MATCH_ID,COMPETITION,SEASON,STADIUM,GAMEWEEK,DAY,HOME_TEAM_XG,AWAY_TEAM_XG,ATTENDANCE,REFEREE,HOME_TEAM_ID,AWAY_TEAM_ID,DATE_TIME,HOME_TEAM_SCORE,AWAY_TEAM_SCORE
0,e62f6e78,ENG-Premier League,2223,Selhurst Park,1,Fri,1.2,1.0,25286,Anthony Taylor,47c64c55,18bb7c10,2022-08-05 20:00:00,0,2
1,877e3193,ENG-Premier League,2223,Vitality Stadium,1,Sat,0.6,0.7,11013,Peter Bankes,4ba7cbea,8602292d,2022-08-06 15:00:00,2,0
2,3a917cee,ENG-Premier League,2223,Goodison Park,1,Sat,0.7,1.5,39254,Craig Pawson,d3fd31cc,cff3d9bb,2022-08-06 17:30:00,0,1
3,6713c1dc,ENG-Premier League,2223,Craven Cottage,1,Sat,1.2,1.2,22207,Andy Madley,fd962109,822bd0ba,2022-08-06 12:30:00,2,2
4,82702941,ENG-Premier League,2223,Elland Road,1,Sat,0.8,1.3,36347,Robert Jones,5bfb9659,8cec06e1,2022-08-06 15:00:00,2,1


In [189]:
df_matches.rename(columns={'game_id': 'MATCH_ID', 'date_time': 'DATE_TIME', 'home_team_id': 'HOME_TEAM_ID', 'away_team_id': 'AWAY_TEAM_ID',
                           'venue': 'STADIUM', 'week': 'GAMEWEEK', 'day': 'DAY','home_team_score': 'HOME_TEAM_SCORE', 
                           'away_team_score':'AWAY_TEAM_SCORE', 'home_xg':'HOME_TEAM_XG','away_xg':'AWAY_TEAM_XG',
                           'attendance':'ATTENDANCE', 'referee': 'REFEREE'}, inplace=True)

In [191]:
df_matches.drop(columns=['date', 'time', 'home_team', 'away_team', 'score'], inplace=True)

In [193]:
df_matches['DATE_TIME'] = df_matches['DATE_TIME'].astype('string')

In [194]:
insert_to_snowflake_DB(['MATCH_ID'], 'MATCHES', df_matches, replace=True)

[12/21/23 14:27:22] INFO     query: [TRUNCATE TABLE MATCHES]                                          ]8;id=480928;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=601201;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

[12/21/23 14:27:23] INFO     query execution done                                                     ]8;id=336303;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=801299;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=87185;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=908755;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=700562;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=41265;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=848231;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=265785;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=581320;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=559652;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=608983;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=393485;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

[12/21/23 14:27:24] INFO     query: [COPY INTO "MATCHES" /*                                           ]8;id=866790;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=607924;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */...]                         

[12/21/23 14:27:25] INFO     query execution done                                                     ]8;id=174537;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=337116;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=743127;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=744551;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

### TBD- Adding shot events data!

In [ ]:
create_shots_query = """
CREATE TABLE IF NOT EXISTS SHOT_EVENTS (
    MATCH_ID VARCHAR(255) PRIMARY KEY,
    SEASON INT,
    TEAM_FBREF_ID VARCHAR(255),
    PLAYER_FBREF_NAME VARCHAR(255),
    MINUTE INT,
    XG DOUBLE,
    PSXG DOUBLE,
    OUTCOME VARCHAR(255),
    DISTANCE INT,
    BODY_PART VARCHAR(255),
    NOTES VARCHAR(255),
    SCA_1_PLAYER_FBREF_NAME VARCHAR(255),
    SCA_1_PLAYER_FBREF_EVENT VARCHAR(255),
    SCA_2_PLAYER_FBREF_NAME VARCHAR(255),
    SCA_2_PLAYER_FBREF_EVENT VARCHAR(255)
);
"""

In [ ]:
fbref_shot_events = fbref.read_shot_events()

In [ ]:
fbref_schedule.reset_index()[['game', 'game_id']]

In [ ]:
df_shots = fbref_shot_events.reset_index().copy()

In [ ]:
df_shots.columns = ['_'.join(col).rstrip('_') for col in df_shots.columns.values]

In [ ]:
df_shots.merge(fbref_schedule.reset_index()[['game', 'game_id']], on='game', how='left')